In [66]:
import tensorflow as tf
import numpy as np
import cv2
import math
from matplotlib import pyplot as plt
from mnist import MNIST
import random

In [113]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, use_bias=False, activation='relu'),
  tf.keras.layers.Dense(10, use_bias=False, activation='softmax')
])
model.summary()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)

model.evaluate(x_test, y_test, verbose=1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               100352    
                                                                 
 dense_5 (Dense)             (None, 10)                1280      
                                                                 
Total params: 101,632
Trainable params: 101,632
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2640 - accuracy: 0.9243
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1149 - accuracy: 0.9660
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0780 - accuracy: 0.9766
Epoch 

[0.08647597581148148, 0.9764999747276306]

In [241]:
#reading dataset image (16x16)
# img = cv2.imread('1.png', 0)
mndata = MNIST('mnist')

# images, labels = mndata.load_training()
def get_rand_train():
    index = random.randrange(0, len(images))  # choose an index ;-)
    print(mndata.display(images[index]))
    imgflat = images[index]
    imgflat = [float(i)/255.0 for i in imgflat] # normalize the array
    img = np.reshape(imgflat, (28, 28))

    #defining time frame of 1s with steps of 5ms
    T = 1;
    dt = 0.005
    time  = np.arange(0, T+dt, dt)
    num_steps = round(T / dt) # 200
    img_width = len(img) # 28
    if img_width != len(img[0]): print("Error! Image is not square") # we assume height and width are equal

    #initializing spike train
    base_train = np.zeros((img_width, img_width, num_steps))
    uni_train = np.copy(base_train)
    poisson_train = np.copy(base_train)
    poisson_isi_train = np.copy(base_train)
    uni_isi = -1
    poisson_isi_isi = -1
    for i in range(img_width):
        for j in range(img_width):
            pixel_value = img[i][j]
            if pixel_value != 0:
                uni_isi = 1 / pixel_value
                poisson_isi_isi = np.ceil(-np.log(random.random())/pixel_value)
            for k in range(num_steps):
                if k % uni_isi == 0:
                    uni_train[i][j][k] = 1
                if random.random() < 1 * pixel_value: # c = 1
                    poisson_train[i][j][k] = 1
                if k % poisson_isi_isi == 0:
                    poisson_isi_train[i][j][k] = 1
    return uni_train, poisson_train, poisson_isi_train

In [78]:
class neuron:
    def __init__(self, min_pot, thresh, leak, weight_arr):
        self.min_pot = min_pot
        self.thresh = thresh
        self.leak = leak
        self.weight_arr = weight_arr
        self.reset()
    
    def step(self, input_arr):
        ret_val = 0
        k = sum([s*self.weight_arr[j] for j, s in enumerate(input_arr)])
        self.pot = self.pot + k + self.leak
        if self.pot >= self.thresh:
            ret_val = self.thresh
            self.pot = self.pot - self.thresh
        if self.pot < self.min_pot:
            self.pot = self.min_pot
        return ret_val
    
    def reset(self):
        self.pot = self.min_pot

In [74]:
class layer:
    def __init__(self, weight_array2d, min_pot, thresh, leak):
        self.neuron_arr = []
        for i in range(len(weight_array2d)):
            self.neuron_arr.append(neuron(min_pot, thresh, leak, weight_array2d[i]))
    
    def step(self, input_arr):
        output_arr = np.zeros((len(self.neuron_arr)))
        for i in range(len(output_arr)):
            output_arr[i] = self.neuron_arr[i].step(input_arr)
        return output_arr
    
    def reset(self):
        for n in self.neuron_arr:
            n.reset()

In [243]:
minimum_potential = 0
threshold_potential = 0.5
leakage = 0

# layer 1 (input layer)
# The spike train itself behaves as the input layer

# layer 2
weights_arr = np.zeros((128, 784))
for i in range(len(weights_arr)):
    for j in range(784):
        weights_arr[i][j] = model.layers[1].weights[0][j][i]
for i in range(len(weights_arr)):
    weights_arr[i] = [(float(m)/np.max(weights_arr))/2 for m in weights_arr[i]]
layer2 = layer(weights_arr, minimum_potential, threshold_potential, leakage)

# layer 3 (output layer)
weights_arr = np.zeros((10, 128))
for i in range(len(weights_arr)):
    for j in range(128):
        weights_arr[i][j] = -model.layers[2].weights[0][j][i]
for i in range(len(weights_arr)):
    weights_arr[i] = [(float(m)/np.max(weights_arr))/2 for m in weights_arr[i]]
layer3 = layer(weights_arr, minimum_potential, threshold_potential, leakage)

In [244]:
#defining time frame of 1s with steps of 5ms
T = 1;
dt = 0.005
time  = np.arange(0, T+dt, dt)
num_steps = round(T / dt) # 200

def run_train(train):
    spike_train = np.array(train).reshape(784, num_steps)
    # spike_train = np.array(poisson_train).reshape(784, num_steps)
    # spike_train = np.array(poisson_isi_train).reshape(784, num_steps)

    # run the model on the given spike train
    output = np.zeros((10))
    layer2.reset()
    layer3.reset()
    for i in range(num_steps):
        m_input = np.zeros(784)
        for j in range(784):
            m_input[j] = spike_train[j][i]
        out_layer2 = layer2.step(m_input)
        out_layer3 = layer3.step(out_layer2)
        for j in range(10):
            output[j] = output[j] + out_layer3[j]
        print(output)
    print("Output Array: ", output)
    print("Max value for: ", np.argmax(output)+1)
    print("Expected output: ", labels[index])

In [247]:
uni_t, poi_t, poi_isi_t = get_rand_train()
run_train(uni_t)


............................
............................
............................
............................
............................
............................
............................
............@@@@@@@@........
...........@@@@@@@@@@@......
..........@@@@....@@@@@.....
.........@@@@@.......@@@....
.......@@@@...........@@@...
.......@@@............@@@...
......@@@.............@@....
.....@@@.............@@@....
.....@@@............@@@.....
.....@@@............@@......
.....@@@...........@@@......
.....@@@..........@@@.......
.....@@@@.....@@@@@.........
......@@@@@@@@@@@@..........
.......@@@@@@@@@@...........
.........@@@................
............................
............................
............................
............................
............................
[0.5 0.  0.  0.  0.5 0.  0.5 0.  0.5 0.5]
[1.  0.5 0.  0.  1.  0.  0.5 0.5 1.  1. ]
[1.5 1.  0.  0.  1.5 0.  0.5 1.  1.5 1.5]
[2.  1.5 0.  0.  2.  0.5 0.5 1.5 2.  2. ]
[2.5 2.  0.  0.  2.

[74.5 69.   0.   0.  74.5 38.5  0.5 74.  74.5 74.5]
[75.  69.5  0.   0.  75.  39.   0.5 74.5 75.  75. ]
[75.5 70.   0.   0.  75.5 39.   0.5 75.  75.5 75.5]
[76.  70.5  0.   0.  76.  39.   0.5 75.5 76.  76. ]
[76.5 70.5  0.   0.  76.5 39.5  0.5 76.  76.5 76.5]
[77.  71.   0.   0.  77.  40.   0.5 76.5 77.  77. ]
[77.5 71.5  0.   0.  77.5 40.   0.5 77.  77.5 77.5]
[78.  72.   0.   0.  78.  40.   0.5 77.5 78.  78. ]
[78.5 72.5  0.   0.  78.5 40.5  0.5 78.  78.5 78.5]
[79.  73.   0.   0.  79.  41.   0.5 78.5 79.  79. ]
[79.5 73.5  0.   0.  79.5 41.   0.5 79.  79.5 79.5]
[80.  74.   0.   0.  80.  41.5  0.5 79.5 80.  80. ]
[80.5 74.5  0.   0.  80.5 41.5  0.5 80.  80.5 80.5]
[81.  75.   0.   0.  81.  42.   0.5 80.5 81.  81. ]
[81.5 75.5  0.   0.  81.5 42.   0.5 81.  81.5 81.5]
[82.  76.   0.   0.  82.  42.5  0.5 81.5 82.  82. ]
[82.5 76.5  0.   0.  82.5 43.   0.5 82.  82.5 82.5]
[83.  77.   0.   0.  83.  43.   0.5 82.5 83.  83. ]
[83.5 77.5  0.   0.  83.5 43.5  0.5 83.  83.5 83.5]
[84.  77.5  

In [165]:
sum(i > 0 for i in layer2.neuron_arr[0].weight_arr)

408

In [222]:
max(sum(el.weight_arr[0:12]) for el in layer3.neuron_arr)

1.449569603201572

In [208]:
max(layer3.neuron_arr[3].weight_arr)

0.6873327670763038